# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [4]:
#1

print(df['tipo_renda'].value_counts())
print(df['educacao'].value_counts())
print(df['estado_civil'].value_counts())
print(df['tipo_residencia'].value_counts())

Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: tipo_renda, dtype: int64
Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: educacao, dtype: int64
Casado      10534
Solteiro     1798
União        1078
Separado      879
Viúvo         711
Name: estado_civil, dtype: int64
Casa             13532
Com os pais        675
Governamental      452
Aluguel            194
Estúdio             83
Comunitário         64
Name: tipo_residencia, dtype: int64


In [5]:
reg_log = smf.ols('''np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos +
                    C(tipo_renda, Treatment('Assalariado')) + C(educacao, Treatment('Secundário')) 
                       + C(estado_civil, Treatment('Casado')) + C(tipo_residencia, Treatment('Casa'))
                       + idade + tempo_emprego + qt_pessoas_residencia''', df).fit()
reg_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Tue, 21 Mar 2023   Prob (F-statistic):               0.00
Time:                        16:57:31   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5264      0.219     29.853      0.000       6.098       6.955
sexo[T.M]                                                       0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]                                        0.0441      0.014      3.119      0.002       0.016       0.072
posse_de_imovel[T.True]                                         0.0829      0.014      5.926      0.000       0.055       0.110
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment('Secundário'))[T.Primário]                0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment('Casado'))[T.União]                  -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment('Casa'))[T.Aluguel]                0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment('Casa'))[T.Com os pais]            0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment('Casa'))[T.Comunitário]           -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment('Casa'))[T.Estúdio]                0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment('Casa'))[T.Governamental]         -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos 

In [6]:
#2

reg_log = smf.ols('''np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos +
                    C(tipo_renda, Treatment('Assalariado')) + C(educacao, Treatment('Secundário')) 
                       + C(estado_civil, Treatment('Casado'))
                       + idade + tempo_emprego + qt_pessoas_residencia''', df).fit()
reg_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Tue, 21 Mar 2023   Prob (F-statistic):               0.00
Time:                        16:57:31   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5315      0.218     29.894      0.000       6.103       6.960
sexo[T.M]                                                       0.7893      0.015     53.999      0.000       0.761       0.818
posse_de_veiculo[T.True]                                        0.0442      0.014      3.125      0.002       0.016       0.072
posse_de_imovel[T.True]                                         0.0819      0.014      5.966      0.000       0.055       0.109
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2219      0.241      0.920      0.357      -0.251       0.695
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1556      0.015     10.433      0.000       0.126       0.185
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.3116      0.241     -1.292      0.196      -0.784       0.161
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao, Treatment('Secundário'))[T.Primário]                0.0097      0.072      0.135      0.893      -0.131       0.150
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1200      0.142      0.845      0.398      -0.159       0.399
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1077      0.014      7.761      0.000       0.080       0.135
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0288      0.032     -0.893      0.372      -0.092       0.034
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3272      0.111      2.942      0.003       0.109       0.545
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2683      0.109      2.464      0.014       0.055       0.482
C(estado_civil, Treatment('Casado'))[T.União]                  -0.0329      0.025     -1.310      0.190      -0.082       0.016
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.3678      0.116      3.182      0.001       0.141       0.594
qtd_filhos                                                     -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                           0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                                                   0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                                           0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:      

O modelo não sofreu grandes mudanças, R² continuou o mesmo.
Nenhum parâmetro mudou muito também.

In [7]:
#3

reg_log = smf.ols('''np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos
                       + C(tipo_renda, Treatment('Assalariado')) 
                       + C(estado_civil, Treatment('Casado'))
                       + idade + tempo_emprego + qt_pessoas_residencia''', df).fit()
reg_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     453.1
Date:                Tue, 21 Mar 2023   Prob (F-statistic):               0.00
Time:                        16:57:32   Log-Likelihood:                -13603.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12411   BIC:                         2.736e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5951      0.219     30.152      0.000       6.166       7.024
sexo[T.M]                                                       0.7819      0.015     53.480      0.000       0.753       0.811
posse_de_veiculo[T.True]                                        0.0535      0.014      3.789      0.000       0.026       0.081
posse_de_imovel[T.True]                                         0.0848      0.014      6.172      0.000       0.058       0.112
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2998      0.242      1.241      0.215      -0.174       0.773
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1655      0.015     11.120      0.000       0.136       0.195
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.2540      0.242     -1.051      0.293      -0.728       0.220
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0769      0.022      3.472      0.001       0.033       0.120
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3241      0.112      2.907      0.004       0.106       0.543
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2623      0.109      2.403      0.016       0.048       0.476
C(estado_civil, Treatment('Casado'))[T.União]                  -0.0361      0.025     -1.431      0.153      -0.086       0.013
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.3588      0.116      3.097      0.002       0.132       0.586
qtd_filhos                                                     -0.2560      0.108     -2.366      0.018      -0.468      -0.044
idade                                                           0.0048      0.001      6.147      0.000       0.003       0.006
tempo_emprego                                                   0.0617      0.001     59.407      0.000       0.060       0.064
qt_pessoas_residencia                                           0.2839      0.108      2.629      0.009       0.072       0.496
==============================================================================
Omnibus:                        1.149   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.563   Jarque-Bera (JB):                1.121
Skew:                           0.021   Prob(JB):                        0.571
Kurtosis:                       3.019   Cond. No.                     2.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+03. This might indicate that there a

In [8]:
df['tipo_renda'] = df.tipo_renda.astype('category')
df['estado_civil'] = df.estado_civil.astype('category')

In [15]:
_df = df[df.tipo_renda!='Bolsista'].copy(deep=True)
_df = _df[_df.tipo_renda!='Pensionista'].copy(deep=True)
_df = _df[_df.estado_civil!='União'].copy(deep=True)


In [16]:
_df['tipo_renda'] = _df.tipo_renda.cat.remove_unused_categories()
print(_df.tipo_renda)

0              Empresário
1             Assalariado
2              Empresário
3        Servidor público
4             Assalariado
               ...       
14993          Empresário
14994          Empresário
14995          Empresário
14997         Assalariado
14998          Empresário
Name: tipo_renda, Length: 11485, dtype: category
Categories (3, object): ['Assalariado', 'Empresário', 'Servidor público']


In [17]:
_df['estado_civil'] = _df.estado_civil.cat.remove_unused_categories()
print(_df.estado_civil)

0        Solteiro
1          Casado
2          Casado
3          Casado
4        Solteiro
           ...   
14993      Casado
14994      Casado
14995    Solteiro
14997      Casado
14998      Casado
Name: estado_civil, Length: 11485, dtype: category
Categories (4, object): ['Casado', 'Separado', 'Solteiro', 'Viúvo']


In [18]:
reg_log = smf.ols('''np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos
                       + C(tipo_renda, Treatment('Assalariado')) 
                       + C(estado_civil, Treatment('Casado'))
                       + idade + tempo_emprego + qt_pessoas_residencia''', _df).fit()
reg_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     532.9
Date:                Tue, 21 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:01:20   Log-Likelihood:                -12581.
No. Observations:               11485   AIC:                         2.519e+04
Df Residuals:                   11472   BIC:                         2.528e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5515      0.229     28.585      0.000       6.102       7.001
sexo[T.M]                                                       0.7942      0.015     52.285      0.000       0.764       0.824
posse_de_veiculo[T.True]                                        0.0493      0.015      3.361      0.001       0.021       0.078
posse_de_imovel[T.True]                                         0.0846      0.014      5.898      0.000       0.056       0.113
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1744      0.016     11.219      0.000       0.144       0.205
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0795      0.023      3.471      0.001       0.035       0.124
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3456      0.117      2.964      0.003       0.117       0.574
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2823      0.114      2.470      0.014       0.058       0.506
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.3833      0.121      3.174      0.002       0.147       0.620
qtd_filhos                                                     -0.2782      0.113     -2.452      0.014      -0.500      -0.056
idade                                                           0.0047      0.001      5.859      0.000       0.003       0.006
tempo_emprego                                                   0.0619      0.001     57.632      0.000       0.060       0.064
qt_pessoas_residencia                                           0.3043      0.113      2.687      0.007       0.082       0.526
==============================================================================
Omnibus:                        1.064   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.587   Jarque-Bera (JB):                1.043
Skew:                           0.022   Prob(JB):                        0.594
Kurtosis:                       3.013   Cond. No.                     2.16e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.16e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

De acordo com os indicadores (ACI e R² ajustado) esse último modelo parece melhor.